In [4]:
import os
from datetime import datetime
import collections

import cv2

import numpy as np
import pandas as pd

import pickle5 as pickle

from pims import PyAVVideoReader

ModuleNotFoundError: No module named 'pims'

In [ ]:
# exttrev
groupHI = ['17202338_a', '17202345_b', '17202338_b', '17202346_b', '17202341_b', '17202341_c', '17202346_c', '17202342_d', '6394837_d','17202341_d', '17202339_d', '6394836_d', '6394841_d']
groupLOW = ['17202339_a', '17202342_a', '17202346_a', '17202341_a', '17202345_a', '17202339_b','17202342_b','17202338_c', '17202345_c', '17202339_c', '17202342_c']

# extt1
groupHI = ['17202339_a', '17202342_a', '17202346_a', '17202341_a','17202345_b', '17202338_b', '17202346_c', '17202345_c', '17202339_c', '17202342_c', '17202342_d', '6394837_d']
groupLOW = ['17202338_a', '17202345_a', '17202346_b', '17202346_b', '17202341_b', '17202339_b','17202342_b','17202341_c', '17202338_c', '17202341_d', '17202339_d', '6394836_d', '6394841_d']

In [ ]:
BEH_LABELS = ['drink', 'eat', 'groom', 'hang', 'sniff', 'rear', 'rest', 'walk', 'eathand']

In [2]:
N_ANIMALS = 24
N_HOURS = 50

FPS = 30
SUMMATION_TIME = 120 # in minutes

In [6]:
postext_datasets = []
postret_datasets = []
postcond_datasets = []
preexposure_datasets = []

In [ ]:
USE_DATASET = "postcond"
LOOK_FOR_BEHAVIOR = "rest"

In [ ]:
base_path = '/media/data_cifs/projects/prj_nih/prj_andrew_holmes/inference/batch_inference/akash'
datasets = os.listdir(base_path)

postext_datasets = [dataset for dataset in datasets if 'postext' in dataset ]
postret_datasets = [dataset for dataset in datasets if 'postret' in dataset ]
postcond_datasets = [dataset for dataset in datasets if 'postcond' in dataset ]
preexposure_datasets = [dataset for dataset in datasets if 'preexposure' in dataset ]

if USE_DATASET == "postext":
    dataset_to_use = postext_datasets
elif USE_DATASET == "postret":
    dataset_to_use = postret_datasets
elif USE_DATASET == "preexposure":
    dataset_to_use = preexposure_datasets
else:
    dataset_to_use = postcond_datasets

all_datasets = [[('FC-A', 'preexposure', 'Trap'), ('FC-B','preexposure'), ('FC-C', 'preexposure'), ('FC-D', 'preexposure')],
            [('FC-A', 'postcond', 'Trap'), ('FC-B','postcond'), ('FC-C', 'postcond'), ('FC-D', 'postcond')],
            [('FC-A', 'postret', 'Trap'), ('FC-B','postret'), ('FC-C', 'postret'), ('FC-D', 'postret')],
            [('FC-A', 'postext', 'Trap'), ('FC-B','postext'), ('FC-C', 'postext'), ('FC-D', 'postext')]]

base_folder_path = "/media/data_cifs/projects/prj_nih/prj_andrew_holmes/inference/batch_inference/akash"

In [29]:
behavior = BEH_LABELS.index(LOOK_FOR_BEHAVIOR)
for folder in dataset_to_use:
    folder_path = base_folder_path+'/'+folder
    print("FOLDER:", folder)

    append_character = folder.split('_')[0][-1]
    append_character = "_"+append_character.lower()

    print(folder, " : ", append_character)

    # Working with postcond FC-A:
    files = os.listdir(folder_path)
    print("Number of files in Current Cohort (1 file = 1 hour of data):", len(files))

    # Sorting the file list to get data in order
    files = sorted(files)

    for file in files:
        camera_id = file.split('cam_')[1].split('-')[0]
        camera_id = camera_id+append_character
        print("Camera ID: ", camera_id)

        year_index = file.find('Y')
        year = int(file[year_index-4:year_index])
        month_index = file.find('M')
        month = int(file[month_index-2:month_index])
        day_index = file.find('D')
        day = int(file[day_index-2:day_index])
        hour_index = file.find('h')
        hour = int(file[hour_index-2:hour_index])
        minute_index = file.find('m')
        minute = int(file[minute_index-2:minute_index])
        second_index = file.find('s')
        second = int(file[second_index-2:second_index])
        date = datetime(year, month, day, hour, minute, second)

        frame_actions = pd.read_csv(folder_path+'/'+file, header=None)

        summary_list = []
        counter = 0
        action_vector = np.zeros(9)
        same_action_counter = 0
        previous_action_prediction = None
        frame_counter = 0
        confidence = 0.95
        window_size = 30*10 # 30 FPS x 10 seconds
        window = [None]*(window_size) 
        window_index = 0
        same_action_counter = 0
        start_time_recorded = False
        start_time = -1
        
        print("Total Number of frames with predicted actions:", len(frame_actions))
        for i in range(len(frame_actions)):
            action_prediction = frame_actions.iat[i,1]
            if window_index < window_size - 1:
                window_index += 1
            else:
                window_index = 0
            window[window_index] = action_prediction
            
            action_distribution = collections.Counter(window)
            max_action = max(action_distribution, key=action_distribution.get)
            try:
                max_action = BEH_LABELS[max_action]
            except: # Required because the list starts with all 'None' values
                pass
            if max_action == previous_action:
                same_action_counter = same_action_counter + 1
            else:
                if same_action_counter >= 300:
                    print("Video:", file, "Action:", BEH_LABELS[action_prediction], "Time Start:", start_time ,"Time End:", frame_counter/30, "seconds")
                same_action_counter = 0
                start_time = frame_counter/30
            previous_action = max_action
            frame_counter += 1

FOLDER: FC-C_1-6_postcond_10-15_10-17
FC-C_1-6_postcond_10-15_10-17  :  _c
Number of files in Current Cohort (1 file = 1 hour of data): 270
Camera ID:  17202339_c
Total Number of frames with predicted actions: 107840
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: sniff Time Start: 8.8 Time End: 36.266666666666666 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: eathand Time Start: 36.266666666666666 Time End: 92.0 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: groom Time Start: 97.73333333333333 Time End: 193.73333333333332 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: eathand Time Start: 200.46666666666667 Time End: 226.73333333333332 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: sniff Time Start: 231.23333333333332 Time End: 321.46666666666664 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202339-0000.csv Action: rest Time Start: 321.4666

Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: groom Time Start: 701.1 Time End: 746.7666666666667 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: sniff Time Start: 755.6 Time End: 819.6333333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: eathand Time Start: 823.6 Time End: 855.9333333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: groom Time Start: 855.9333333333333 Time End: 951.3 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: groom Time Start: 959.1 Time End: 1227.6666666666667 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: rest Time Start: 1227.6666666666667 Time End: 1266.7 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: groom Time Start: 1266.7 Time End: 1353.0333333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202342-0000.csv Action: groom Time Start: 1362

Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: sniff Time Start: 1360.5666666666666 Time End: 1574.3666666666666 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: rest Time Start: 1574.3666666666666 Time End: 1590.8333333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: eat Time Start: 1590.8333333333333 Time End: 2059.233333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: rest Time Start: 2059.233333333333 Time End: 2071.0333333333333 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: sniff Time Start: 2071.0333333333333 Time End: 2430.766666666667 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: rest Time Start: 2430.766666666667 Time End: 2441.366666666667 seconds
Video: video_2018Y_10M_15D_10h_15m_58s_cam_17202346-0000.csv Action: sniff Time Start: 2441.366666666667 Time End: 2857.4333333333334 seconds
Video: vid

Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: rest Time Start: 2861.2 Time End: 2951.4333333333334 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: rest Time Start: 2954.266666666667 Time End: 3037.8333333333335 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: sniff Time Start: 3037.8333333333335 Time End: 3059.9 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: rest Time Start: 3063.266666666667 Time End: 3103.0333333333333 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: sniff Time Start: 3103.0333333333333 Time End: 3121.6 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: rest Time Start: 3121.6 Time End: 3158.4666666666667 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv Action: sniff Time Start: 3158.4666666666667 Time End: 3178.233333333333 seconds
Video: video_2018Y_10M_15D_11h_15m_58s_cam_17202341-00

Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: groom Time Start: 1443.4666666666667 Time End: 1475.7666666666667 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: sniff Time Start: 1475.7666666666667 Time End: 1493.7333333333333 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: sniff Time Start: 1500.0666666666666 Time End: 1522.4666666666667 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: sniff Time Start: 1543.7666666666667 Time End: 1569.6 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: sniff Time Start: 1573.7666666666667 Time End: 1589.0 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: groom Time Start: 1589.0 Time End: 1606.5666666666666 seconds
Video: video_2018Y_10M_15D_11h_15m_59s_cam_17202346-0000.csv Action: groom Time Start: 1614.9666666666667 Time End: 1633.3333333333333 seconds
Video: video_2018Y_10M_15D_11h_15m_

...

Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: sniff Time Start: 1608.2 Time End: 1667.4 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: walk Time Start: 1667.4 Time End: 1686.3 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: eat Time Start: 1699.9 Time End: 1734.6 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: sniff Time Start: 1734.6 Time End: 1747.3666666666666 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: eat Time Start: 1747.3666666666666 Time End: 1767.7333333333333 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: eat Time Start: 1767.7333333333333 Time End: 1785.0666666666666 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: walk Time Start: 1789.9333333333334 Time End: 1805.3 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202339-0000.csv Action: hang Time Start: 1805.3 Time End: 

Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: sniff Time Start: 221.73333333333332 Time End: 233.73333333333332 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 233.73333333333332 Time End: 281.93333333333334 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: hang Time Start: 290.76666666666665 Time End: 325.5 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: hang Time Start: 325.5 Time End: 336.93333333333334 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: walk Time Start: 340.03333333333336 Time End: 357.1333333333333 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: sniff Time Start: 364.6333333333333 Time End: 378.73333333333335 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 378.73333333333335 Time End: 404.03333333333336 seconds
Video: video_2018Y_10M_16D_10h_35m

Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 2339.1666666666665 Time End: 2474.6666666666665 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 2483.733333333333 Time End: 2542.8 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 2547.4 Time End: 2689.8333333333335 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: sniff Time Start: 2689.8333333333335 Time End: 2700.866666666667 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: sniff Time Start: 2717.6666666666665 Time End: 2727.866666666667 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: eat Time Start: 2727.866666666667 Time End: 2875.4 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202341-0000.csv Action: hang Time Start: 2876.366666666667 Time End: 2891.233333333333 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_1720234

Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: sniff Time Start: 1078.4 Time End: 1096.3 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: walk Time Start: 1096.3 Time End: 1127.4 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: eat Time Start: 1145.6 Time End: 1178.1 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: sniff Time Start: 1182.4333333333334 Time End: 1234.3 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: eat Time Start: 1234.3 Time End: 1252.9 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: hang Time Start: 1252.9 Time End: 1287.8666666666666 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: walk Time Start: 1287.8666666666666 Time End: 1305.2 seconds
Video: video_2018Y_10M_16D_10h_35m_40s_cam_17202342-0000.csv Action: walk Time Start: 1311.0666666666666 Time End: 1340.8333333333333 seco

KeyboardInterrupt: 

In [ ]:
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 832.53 - 887.06 seconds -> Rear
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 917.3 - 948.4 seconds -> Eat
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 951.1 - 1081.6 seconds -> Hang 29:16 - 29:28
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 1083.8 - 1097.7 seconds -> Sniff
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 1510.76 - 1587.33 seconds -> Groom
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 1726.06 - 1743.26 seconds -> Drink
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 1802.93 - 1844.73 seconds -> Walk
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 2032.63 - 2692.1 seconds -> Eathand
# video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.csv 2861.2 - 2951.43 seconds -> Rest

# 5 minute video: 28minutes 15seconds - 33minutes 35seconds

In [7]:
file = "video_2018Y_10M_15D_11h_15m_58s_cam_17202341-0000.mp4"
files_to_send = "/media/data_cifs/nih/files_to_send"
folder_of_video = "/FC-C_1-6_postcond_10-15_10-17/" # Found manually using -> $ find *<video_name>*
video = PyAVVideoReader(files_to_send + folder_of_video + file)

In [11]:
vidcap = cv2.VideoCapture(files_to_send + folder_of_video + file)
success, image = vidcap.read()
frame_count = 0
action_class = "sniff3"
while success:
    success, image = vidcap.read()
    if frame_count >= 84600 and frame_count <= 84900:
        cv2.imwrite("VideoActions/" + action_class+"/frame%d.jpg" % frame_count, image)
    frame_count += 1
    if frame_count % 500 == 0:
        print("Processed frame:", frame_count)

Application has requested 20 threads. Using a thread count greater than 16 is not recommended.


Processed frame: 500
Processed frame: 1000
Processed frame: 1500
Processed frame: 2000
Processed frame: 2500
Processed frame: 3000
Processed frame: 3500
Processed frame: 4000
Processed frame: 4500
Processed frame: 5000
Processed frame: 5500
Processed frame: 6000
Processed frame: 6500
Processed frame: 7000
Processed frame: 7500
Processed frame: 8000
Processed frame: 8500
Processed frame: 9000
Processed frame: 9500
Processed frame: 10000
Processed frame: 10500
Processed frame: 11000
Processed frame: 11500
Processed frame: 12000
Processed frame: 12500
Processed frame: 13000
Processed frame: 13500
Processed frame: 14000
Processed frame: 14500
Processed frame: 15000
Processed frame: 15500
Processed frame: 16000
Processed frame: 16500
Processed frame: 17000
Processed frame: 17500
Processed frame: 18000
Processed frame: 18500
Processed frame: 19000
Processed frame: 19500
Processed frame: 20000
Processed frame: 20500
Processed frame: 21000
Processed frame: 21500
Processed frame: 22000
Processe